# Solving a fisheries optimization in Python

This notebook will introduce you both to a classical Optimal Control problem defined for a fishery but also will show you how to use GEKKO, an open-source dynamic optimization toolkit.

Unlike if you have taken my big-data course, this will be run primarily through an online, cloud-hosted Docker image. You don't need to know what this means other than to (hopefully) appreciate that the only software you will require is Chrome (or a modern equivilent).

## Import required packages

These lines will import the packages into python. On the line import numpy as np, the np is just a shortened version of the name that we will assign the package to so we don't have to write out numpy each time. Same for plt.

In [ ]:
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt


Next we will create the GEKKO model and save it as the variable m. Python is an "object oriented programming language". Understanding this fully is outside the domain of this course, but for our purposes here, just know that when we called GEKKO(), it created a big, complex object and saved it in m. We will use this object below to store input parameters and it will also give us the key functions (called "methods" when they are attached to an object) we need to run the optimization.

In [ ]:
m = GEKKO()
print('Printing out the object, which will tell us how it wants to be expressed:')
print(m)

Now that the object is created, we can set it's attributes, which are denoted via a "dot", as in m.attrbute. We will also be setting the key constants for our model (refer to lecture notes).

In [ ]:
# time points
n=501
m.time = np.linspace(0,10,n)

# constants
p = 1 # A price index
c = 17.5 # Cost
r = 0.71 # Intrinsic growth rate (from logistic function)
k = 80.5 # Carrying capacity
Y_max = 20 # Maximum effort one could expend
q = 1 # An index of fishing "effort" per the Shaeffer model design.

print('Time attribute: ' + str(m.time))

The above variables were constant and so were defined just as python variables. Now, however, we need to create our more complex "state" and "contorl" variables. GEKKO gives us some functions to create these, such as m.MV(), which is a "Manipulated Variables" in GEKKO's terminology but what we have been calling control variable. When creating this u function, we also give it a few input parameters such as the lower-bound and upper-bound.

Create the control variable:

In [ ]:
# fishing rate
y = m.MV(value=1,lb=0,ub=1)
y.STATUS = 1
y.DCOST = 0

Now we're going to create a GEKKO variable, which is the thing that the solver will be solving to meet the constraints.

In [ ]:
# fish population
x = m.Var(value=70)

Define our equation of motion as a GEKKO Equation. Note that the this expression has both python variables (e.g. the constants from above) and GEKKO variables that will be changing as we solve it.

TASK! Define the fish population equation of motion using our notes on the logistic growth curve. Fill in the MISSING_PART with a mathematical representation of the model using the coefficients defined above.

In [ ]:
MISSING_PART = None
m.Equation(x.dt() == MISSING_PART)

The objective function itself (profit), is defined as a value J, but then we also will be defining a Final objective, Jf, which we will use in the solution.

In [ ]:
# objective (profit)
J = m.Var(value=0)
# final objective
Jf = m.FV()
Jf.STATUS = 1

In order to connect the different functions, we are going to use the m.Connection method. Here we connect the final objective function to the end position of our objective J.

TASK! Like above, fill in the equation for J.dt() by replacing the MISSING_PART with the Schaeffer equation of benefit using the coefficients defined above.

In [ ]:
m.Connection(Jf,J,pos2='end')

m.Equation(J.dt() == MISSING_PART)

# Also define what exactly we will be Maximizing in the model
m.Maximize(Jf)

GEKKO has many solution methods which we set below. Notice that here we're telling GEKKO that we want this solved via optimal control methods and that it will use a particular solvew (IPOPT).

In [ ]:
# options
m.options.IMODE = 6  # optimal control
m.options.NODES = 3  # collocation nodes
m.options.SOLVER = 3 # solver (IPOPT)

Now, after all this hard work, we can finally call the solve() method from our object.

In [ ]:
# solve optimization problem
m.solve()

# print profit
print('Optimal Profit: ' + str(Jf.value[0]))

Now just plot the results!

In [ ]:
# plot results
plt.figure(figsize=(8, 6), dpi=80)

plt.subplot(2,1,1)
plt.plot(m.time,J.value,'r--',label='profit')
plt.plot(m.time[-1],Jf.value[0],'ro',markersize=10,
         label='final profit = '+str(Jf.value[0]))
plt.plot(m.time,x.value,'b-',label='fish population')
plt.ylabel('Value')
plt.legend()
plt.subplot(2,1,2)
plt.plot(m.time,y.value,'k.-',label='fishing rate')
plt.ylabel('Rate')
plt.xlabel('Time (yr)')
plt.legend()
plt.show()
